In [1]:
!pip install openai

!pip install gym

!pip install requests

!pip install beautifulsoup4

You should consider upgrading via the '/Users/tianyixu/Documents/github_projects/tree-of-thought-llm/tot_venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/tianyixu/Documents/github_projects/tree-of-thought-llm/tot_venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/tianyixu/Documents/github_projects/tree-of-thought-llm/tot_venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/tianyixu/Documents/github_projects/tree-of-thought-llm/tot_venv/bin/python3 -m pip install --upgrade pip' command.


# Setup

In [ ]:
import os
import openai
 
openai.api_key = os.environ["OPENAI_API_KEY"]

# Takes a prompt and an optional stop sequence (default: ["\n"]), which signals where the model should stop generating text.
def llm(prompt, stop=["\n"]):
    
    # Sends the prompt to OpenAI’s gpt-3.5-turbo-instruct model
    response = openai.Completion.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      temperature=0,            # Low randomness, aiming for more predictable responses.
      max_tokens=100,
      top_p=1,                  # Selective so model model is selective
      frequency_penalty=0.0,    # No penalty for anything
      presence_penalty=0.0,
      stop=stop
    )
    return response["choices"][0]["text"]

## Wikienv:
A library for interacting with Wikipedia-based environments

## 

In [2]:
import wikienv, wrappers

# Initializes a WikiEnv instance
env = wikienv.WikiEnv()
env = wrappers.HotPotQAWrapper(env, split="dev")        # Wraps env with HotPotQAWrappe
env = wrappers.LoggingWrapper(env)                      # Wraps env again with LoggingWrapper, adding logging functionality to monitor and track the actions


# Attempts to execute an action within the environment
def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

/Users/tianyixu/Documents/github_projects/tree-of-thought-llm/tot_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# ReAct

## Brief Explanation:

Run a question-answering (QA) task using a prompt-based approach that simulates multi-step reasoning with the help of an LLM (like GPT) and an interactive environment (env). (Here we are using Wikienv)

## Prompts_naive.json explaiend:

prompts_naive.json is a Prompting Guide, it contains structured example prompts with pre-defined questions and multi-step answers (using Thought → Action → Observation).
These examples help prime the model to handle new questions in a similar structured way -> -> it helps set expectations for how the model should reason through questions.

Why Use prompts_naive.json?
Without examples to show the structure (like Thought → Action → Observation), the model might not naturally break down complex questions or simulate the reasoning steps effectively.
prompts_naive.json essentially acts as a scaffolding to guide the model’s responses, making the evaluation more reliable and coherent.

In [3]:
import json
import sys


#----------------------------------------------------------------------------------------------------
# Loads the prompts_naive.json file, which includes sample QA interactions, into prompt_dict.
#----------------------------------------------------------------------------------------------------
folder = './src/reAct/prompts/'
prompt_file = 'prompts_naive.json'
with open(folder + prompt_file, 'r') as f:
    prompt_dict = json.load(f)


#----------------------------------------------------------------------------------------------------
# This setup combines instructions with webthink_examples, which provides a step-by-step Thought → Action → Observation format
#----------------------------------------------------------------------------------------------------
webthink_examples = prompt_dict['webthink_simple6']
instruction = """Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.
Here are some examples.
"""
webthink_prompt = instruction + webthink_examples

# prompts_naive.json is just the prompting framework that shows the model how to approach and answer each question in a structured format.



#  Executes the QA task within an environment (env) using the structured Thought → Action → Observation process.

# Simulates Multi-Step Reasoning: Each call to webthink aims to answer a question by breaking it down into sequential steps, each involving:
#	Thought: Generates a “Thought” based on the current prompt.
#	Action: The model chooses an 'action' based on its thought, such as searching for information or finishing with an answer.
#	Observation: Receives an “Observation” from environemnt as feedback, which the model uses to inform the next step.

def webthink(idx=None, prompt=webthink_prompt, to_print=True):

    # First: Reset the environment for the specific question identified by idx
    question = env.reset(idx=idx)
    if to_print:
        print(idx, question)
    prompt += question + "\n"
    n_calls, n_badcalls = 0, 0


    # For loop inside the webthink function is where the model iteratively generates thoughts and actions to answer a question in a structured way
    for i in range(1, 8):
        n_calls += 1

        # First Step – Generating Initial Thought:
        thought_action = llm(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:

            # Strip action and thought from each other
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = llm(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()


        # Performing Action and Generating Observation:
            # action is executed in the environment (env) through the step function
            # Observation is the feedback or result that the environment (env) returns after the model takes an action.
            # Reward provides a numeric score for the action taken by the model
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')

        # Adds the Thought, Action, and Observation sequence for each step to prompt, allowing the model to build on its prior responses.
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print:
            print(step_str)

        # Ending the Loop Early if model decides its good
        if done:
            break
        # With each Observation added to the prompt, the model has more information to base its next Thought on.
    
    # If the loop completes its maximum iterations (7 in this case) without done being True, the function calls a finish[] action to explicitly end the task.
    if not done:
        obs, r, done, info = step(env, "finish[]")
    if to_print:
        print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})

    
    return r, info

In [6]:
import random
import time
idxs = list(range(7405))
random.Random(233).shuffle(idxs)

# Stores the exact match scores (em) of each webthink run (indicating whether the answer was entirely correct)
rs = []
infos = []                  # Collects detailed info from each webthink call.
old_time = time.time()


for i in idxs[:500]:
    r, info = webthink(i, to_print=True)
    rs.append(info['em'])                   # info['em'] representing the exact match score (1 if the answer is correct, 0 otherwise)
    infos.append(info)


	#	sum(rs): Counts the total number of correct answers (exact matches).
	#	len(rs): Tracks how many questions have been processed.
	#	sum(rs) / len(rs): Calculates the exact match rate (percentage of correct answers).
	#	(time.time() - old_time) / len(rs): Computes the average time per question by dividing the elapsed time by the number of questions processed.
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

3687 Question: What movie did actress Irene Jacob complete before the American action crime thriller film directed by Stuart Bird?


AttributeError: module 'openai' has no attribute 'completion'